<a href="https://colab.research.google.com/github/justin95214/logistics_colab_code/blob/main/BLOCK_FLEX1_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from itertools import *
import collections
import pandas as pd
import io
from google.colab import files

In [ ]:
# 데이터.xlsx 파일 읽기
data = files.upload()

Saving 데이터.xlsx to 데이터.xlsx


In [ ]:
data = "C:/Users/NSE/Desktop/데이터.xlsx"
location_table = pd.DataFrame()
value_table = pd.DataFrame()
divide_table = pd.DataFrame()
url = "데이터.xlsx"


# Activitiy작업량 시트 데이터 가져오기
try:
    Activity_table = pd.read_excel(io.BytesIO(condition[url]), sheet_name= 'Activity작업수량', skiprows= 1)
except:
    Activity_table = pd.read_excel('/content/데이터.xlsx', sheet_name= 'Activity작업수량', skiprows= 1)

# Location거리 시트 데이터 가져오기
try:
    Location_table = pd.read_excel(io.BytesIO(condition[url]), sheet_name= 'Location거리', skiprows= 1)
except:
    Location_table = pd.read_excel('/content/데이터.xlsx', sheet_name= 'Location거리', skiprows= 1)

#  시트 데이터 가져오기
try:
    divide_table = pd.read_excel(io.BytesIO(condition[url]), sheet_name= 'Activity작업분할표')
except:
    divide_table = pd.read_excel('/content/데이터.xlsx', sheet_name= 'Activity작업분할표')

In [ ]:
print("Activity 작업 수량 테이블 그룹핑 전 :")
Activity_table

Activity 작업 수량 테이블 그룹핑 전 :


,From To,입 하,하 차,검수/검품,입고/적치,재고조사,주문마감,피킹 작업,유통기공,출고 확정,상차 출하
0,입하,-,100,30,40,NaN,NaN,30,NaN,NaN,NaN
1,하차,10,-,40,50,NaN,NaN,NaN,NaN,NaN,NaN
2,검수/검품,NaN,NaN,-,30,NaN,NaN,40,NaN,NaN,NaN
3,입고/적치,NaN,NaN,20,-,20,NaN,80,NaN,NaN,15
4,재고조사,NaN,NaN,NaN,10,-,NaN,20,NaN,NaN,10
5,주문마감,NaN,NaN,NaN,NaN,NaN,-,15,30,NaN,40
6,피킹 작업,NaN,NaN,NaN,NaN,NaN,10,-,10,NaN,60
7,유통기공,NaN,NaN,NaN,NaN,NaN,30,10,-,40,60
8,출고 확정,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,-,10
9,상차 출하,NaN,NaN,NaN,NaN,20,NaN,10,10,10,-


In [ ]:
print("Location 작업 테이블 :")
Location_table

Location 작업 테이블 :


,From To,A,B,C,D
0,A,-,20,40,60
1,B,20,-,30,50
2,C,40,30,-,40
3,D,60,50,40,-


In [ ]:
print("그룹핑 작업 테이블 :")
divide_table

그룹핑 작업 테이블 :


,작업,Activity그룹
0,입하,Activity 1
1,하차,Activity 1
2,검수/검품,Activity 1
3,입고/적치,Activity 2
4,재고조사,Activity 2
5,주문마감,Activity 3
6,피킹 작업,Activity 3
7,유통기공,Activity 4
8,출고 확정,Activity 4
9,상차 출하,Activity 4


In [ ]:
def Block_caluate_cost(location_table : list, value_table: list, Location : list):
    # 테이블의 row, col 값
    rows = len(location_table[0])
    cols = len(location_table)
    
    # 후보 값들이 저장될 리스트
    cal_set = []
    location_set = []

    # 작업 과정 리스트 
    work_list = [ i for i in range(1,cols+1)]
    # [1,2,3,4]

    # 작업 과정 순서 경우의 수 리스트
    sorted_work_list = list(permutations(work_list, len(work_list)))
    

    #작업과정 순서 테이블 인덱스가 담길 테이블 
    arr = [[0 for j in range(rows)] for i in range(cols)]

    # case table
    for m, case in enumerate(sorted_work_list):
        #작업과정 순서 테이블
        new_value_table = [[0 for j in range(rows)] for i in range(cols)]

        # 작업과정 순서 테이블 인덱스를 값으로 가지고 있는 테이블 생성

        for col, work_case_col in enumerate(case):
            for row, work_case_row in enumerate(case):
                arr[col][row] = str(work_case_col) +"/" + str(work_case_row)


        print("Activity작업량 인덱스 테이블 : "+ str(m+1) +"번째 경우의 수")
        for i in range(rows):
            print(arr[i])


        # 작업과정 순서 인덱스에 맞는 소요값을 입력 
        for c in range(cols):
            for r in range(rows):
                idx = arr[c][r].split("/")
                new_value_table[c][r] = value_table[int(idx[0]) - 1][int(idx[1])- 1]

        print("Activity 작업량 수량 테이블 : 경우의 수")
        for i in range(rows):
            print(new_value_table[i])

        print("\n")
        # 2차원 리스트 1차원으로 변경
        new_value_table_flatten = np.concatenate(new_value_table).tolist()
        location_table_flatten = np.concatenate(location_table).tolist()

        location_set.append(list(Location))

        # sumproduct 함수 연산
        mult_list = [new_value_table_flatten[i] * location_table_flatten[i] for i in range(len(location_table_flatten))]
        #print(mult_list)
        cal_set.append(sum(mult_list))


    return location_set, cal_set, sorted_work_list  

In [ ]:
# 마스터 키를 활용한 Activity 카운팅
def master_group(cols: int, data: pd.DataFrame):

    data_columns = data.columns
    Activity_list = data[data_columns[1]].values.tolist()
    
    ab = collections.Counter(Activity_list)
    print("group counting : ",ab)

    case =list(ab.values())
    return case

In [ ]:
def func(cols: int, data: pd.DataFrame, case : list):

    # columns 그룹 분할 작업
    cols_list = data.columns
    tmp = []
    for l in case:

        each_columns_list = []
        for each in range(l):
            each_columns_list.append(cols_list[0])
            cols_list = cols_list[1:]
        tmp.append(each_columns_list)


    # row 그룹 분할 작업
    cache_data = data
    tmp1 = []
    l_sum = 0
    for l in case:
        l_sum+=l
        tmp1.append([l_sum - l, l_sum])

    candid_table = []

    arr = [[0 for j in range(cols)] for i in range(cols)]

    for idx_c, each_col in enumerate(tmp):
        for idx_r, each_row in  enumerate(tmp1):
            temp_df = data[each_col]
            temp_df = temp_df.iloc[each_row[0] : each_row[1]]

            temp_df_list = temp_df.values.tolist()

            del temp_df
            temp_df_list_flatten = np.concatenate(temp_df_list).tolist()
            sum_tmp = sum(temp_df_list_flatten)

            #print(">>",sum_tmp)
            arr[idx_c][idx_r] = sum_tmp

            if idx_c == idx_r:
                arr[idx_c][idx_r] = 0


    candid_table.append(arr)
    del arr

    return candid_table

In [ ]:
# 첫 컬럼 제거
Activity_table= Activity_table.drop(Activity_table.columns[0], axis=1)
Location_table= Location_table.drop(Location_table.columns[0], axis=1)

# - 값 0 으로 대체
Activity_table = Activity_table.replace('-', 0)
Location_table = Location_table.replace('-', 0)

# NaN값 0 으로 대체
Activity_table = Activity_table.replace(np.NaN, 0)
Location_table = Location_table.replace(np.NaN, 0)

saved_value_table = Activity_table
Location = Location_table.columns

In [ ]:
# 데이터프레임 리스트화
Activity_table = Activity_table.values.tolist()
Location_table = Location_table.values.tolist()

Act_length = len(Activity_table)
Loc_length = len(Location_table)

In [ ]:
tmp_df =pd.DataFrame()

if Act_length == Loc_length:
    print("Activity작업량과 Location 작업량이 동일할 떄: ", Act_length ,"x", Loc_length)
    location_set, candidate_cost_list, idx_list = Block_caluate_cost(Activity_table, Location_table, Location)
    
else:
    print("Activity작업량과 Location 작업량이 다를 떄: ", Act_length ,"x", Loc_length)
    divided_list = master_group(Loc_length, divide_table)
    candid_table = func(Loc_length, saved_value_table, divided_list)

    idx_list = [ 'Activity '+str(i) for i in range(1, len( candid_table[0])+1 )]
  
    tmp_df =pd.DataFrame(candid_table[0], columns= idx_list) 
    tmp_df = tmp_df.T
    tmp_df.columns = idx_list
    location_set, candidate_cost_list, idx_list = Block_caluate_cost(candid_table[0], Location_table, Location)

Activity작업량과 Location 작업량이 다를 떄:  10 x 4
group counting :  Counter({'Activity 1': 3, 'Activity 4': 3, 'Activity 2': 2, 'Activity 3': 2})
Activity작업량 인덱스 테이블 : 1번째 경우의 수
['1/1', '1/2', '1/3', '1/4']
['2/1', '2/2', '2/3', '2/4']
['3/1', '3/2', '3/3', '3/4']
['4/1', '4/2', '4/3', '4/4']
Activity 작업량 수량 테이블 : 경우의 수
[0, 20, 40, 60]
[20, 0, 30, 50]
[40, 30, 0, 40]
[60, 50, 40, 0]


Activity작업량 인덱스 테이블 : 2번째 경우의 수
['1/1', '1/2', '1/4', '1/3']
['2/1', '2/2', '2/4', '2/3']
['4/1', '4/2', '4/4', '4/3']
['3/1', '3/2', '3/4', '3/3']
Activity 작업량 수량 테이블 : 경우의 수
[0, 20, 60, 40]
[20, 0, 50, 30]
[60, 50, 0, 40]
[40, 30, 40, 0]


Activity작업량 인덱스 테이블 : 3번째 경우의 수
['1/1', '1/3', '1/2', '1/4']
['3/1', '3/3', '3/2', '3/4']
['2/1', '2/3', '2/2', '2/4']
['4/1', '4/3', '4/2', '4/4']
Activity 작업량 수량 테이블 : 경우의 수
[0, 40, 20, 60]
[40, 0, 30, 40]
[20, 30, 0, 50]
[60, 40, 50, 0]


Activity작업량 인덱스 테이블 : 4번째 경우의 수
['1/1', '1/3', '1/4', '1/2']
['3/1', '3/3', '3/4', '3/2']
['4/1', '4/3', '4/4', '4/2']
['2/1', '2/3', '2/

In [ ]:
print("그룹핑된 Activity 테이블 :")
tmp_df

그룹핑된 Activity 테이블 :


,Activity 1,Activity 2,Activity 3,Activity 4
Activity 1,0.0,120.0,70.0,0.0
Activity 2,20.0,0.0,100.0,25.0
Activity 3,0.0,0.0,0.0,140.0
Activity 4,0.0,20.0,50.0,0.0


In [ ]:
result_df = pd.DataFrame([location_set, idx_list, candidate_cost_list])
result_df = result_df.T
result_df.columns = columns =['Location작업','Activity작업','비용']
result_df

,Location작업,Activity작업,비용
0,"[A, B, C, D]","(1, 2, 3, 4)",18450.0
1,"[A, B, C, D]","(1, 2, 4, 3)",20950.0
2,"[A, B, C, D]","(1, 3, 2, 4)",21300.0
3,"[A, B, C, D]","(1, 3, 4, 2)",24650.0
4,"[A, B, C, D]","(1, 4, 2, 3)",22300.0
5,"[A, B, C, D]","(1, 4, 3, 2)",23150.0
6,"[A, B, C, D]","(2, 1, 3, 4)",19200.0
7,"[A, B, C, D]","(2, 1, 4, 3)",21700.0
8,"[A, B, C, D]","(2, 3, 1, 4)",22800.0
9,"[A, B, C, D]","(2, 3, 4, 1)",24900.0


In [ ]:
print("최소 비용 : ",min(candidate_cost_list))

#업로드 엑셀 삭제
!rm /content/데이터.xlsx

최소 비용 :  17700.0
